# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [260]:
problem = '''Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.  
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment. 

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01. 
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r.'''

## 2. Ask for parameters

In [284]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [285]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

*  **x<sub>j</sub>:** Binary variable indicating whether to invest in strategy j (1) or not (0), where j = 1, 2, ..., 10.
* **y<sub>i</sub>:** Binary variable indicating whether budget scenario i occurs (1) or not (0), where i = 1, 2, 3, 4. 


# 2. Ask for objective

In [286]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [287]:
Markdown(response2.text)

$$\max  \sum_{t=1}^{T} \frac{1}{T} [\sum_{j=1}^{10}  (R_{tj} * x_j)] - r *  \sum_{t=1}^{T} \frac{1}{T}  [(\sum_{j=1}^{10}  (R_{tj} * x_j) - \sum_{t=1}^{T} \frac{1}{T} [\sum_{j=1}^{10}  (R_{tj} * x_j)])^2]  $$ 


# 3. Ask for constraints

In [288]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [289]:
Markdown(response3.text)

## Constraints:

1. **Budget Constraint:**
   -  For each scenario i: 
      ∑<sub>j=1</sub><sup>10</sup> A<sub>j</sub> * x<sub>j</sub>  ≤  B<sub>i</sub> * y<sub>i</sub>  
      where B<sub>i</sub> represents the budget for scenario i (1000, 1100, 900, 1200).

2. **Scenario Selection:** Only one budget scenario can occur:
   - ∑<sub>i=1</sub><sup>4</sup> y<sub>i</sub> = 1

3. **Investment Uniqueness:** Invest in each strategy only once:
   - x<sub>j</sub> ∈ {0, 1} for j = 1, 2, ..., 10

4. **Scenario Probability:** Ensure the probability of exceeding the budget is within the tolerable limit:
   - 0.55y<sub>1</sub> + 0.4y<sub>2</sub> + 0.04y<sub>3</sub> + 0.01y<sub>4</sub>  ≤  0.4 

5. **Scenario Binary:**
   - y<sub>i</sub> ∈ {0, 1} for i = 1, 2, 3, 4 


## 3. Generate the pyomo code

In [290]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [291]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo
import random

# Sample data for T and R_tj (You'll need to replace these with your actual data)
T = 5 
R = {(t,j): random.randint(50, 150) for t in range(1, T+1) for j in range(1, 11)}  
r = 0.05  # Sample risk aversion coefficient

# Model
model = pyo.ConcreteModel()

# Sets
model.J = pyo.RangeSet(1, 10)  # Strategies
model.I = pyo.RangeSet(1, 4)   # Budget scenarios
model.T = pyo.RangeSet(1, T)   # Time periods 

# Parameters
model.A = {j: random.randint(50, 150) for j in model.J}  # Investment cost for each strategy (sample data)
model.B = {1: 1000, 2: 1100, 3: 900, 4: 1200}  # Budget for each scenario
model.R = pyo.Param(model.T, model.J, initialize=R) # Returns
model.r = pyo.Param(initialize=r)

# Variables
model.x = pyo.Var(model.J, within=pyo.Binary)  # Investment decisions
model.y = pyo.Var(model.I, within=pyo.Binary)  # Budget scenario occurrence

# Objective function
def objective_rule(model):
    expected_return = sum(1/T * sum(model.R[t, j] * model.x[j] for j in model.J) for t in model.T)
    risk = model.r * sum(1/T * (sum(model.R[t, j] * model.x[j] for j in model.J) - expected_return)**2 for t in model.T)
    return expected_return - risk
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def budget_constraint(model, i):
    return sum(model.A[j] * model.x[j] for j in model.J) <= model.B[i] * model.y[i]
model.budget_constraint = pyo.Constraint(model.I, rule=budget_constraint)

def scenario_selection(model):
    return sum(model.y[i] for i in model.I) == 1
model.scenario_selection = pyo.Constraint(rule=scenario_selection)

def probability_constraint(model):
    return 0.55*model.y[1] + 0.4*model.y[2] + 0.04*model.y[3] + 0.01*model.y[4] <= 0.4
model.probability_constraint = pyo.Constraint(rule=probability_constraint)

# Solve
solver = pyo.SolverFactory('gurobi')  # Choose a solver you have installed
solver.solve(model)

# Output
print("Optimal Solution:")
for j in model.J:
    print(f"x[{j}] = {pyo.value(model.x[j])}")
for i in model.I:
    print(f"y[{i}] = {pyo.value(model.y[i])}")
print(f"Objective Value: {pyo.value(model.objective)}") 
```

**Explanation and Sample Data:**

1. **Sample Data:**
   - `T = 5`:  Assume 5 time periods. You'll replace this with your actual number of periods.
   - `R = {(t,j): random.randint(50, 150) for t in range(1, T+1) for j in range(1, 11)}`: Generates random return values between 50 and 150 for each time period and strategy. Replace this with your actual return data.
   - `r = 0.05`: Sample risk aversion coefficient. Adjust based on your preferences.
   - `model.A = {j: random.randint(50, 150) for j in model.J}`: Randomly generated investment costs for each strategy. 

2. **Code Structure:**
   - **Import Necessary Libraries:** Import `pyomo` for modeling and a solver library (like `gurobi`).
   - **Create a Model:**  `model = pyo.ConcreteModel()` initializes an optimization model.
   - **Define Sets:** Define sets for strategies (`J`), budget scenarios (`I`), and time periods (`T`).
   - **Define Parameters:** 
     - Use `pyo.Param()` to define parameters like investment costs (`A`), budget levels (`B`), returns (`R`), and the risk aversion coefficient (`r`).
   - **Define Variables:**
     - Use `pyo.Var(..., within=pyo.Binary)` to define binary variables for investment decisions (`x`) and budget scenario occurrence (`y`).
   - **Define the Objective Function:**
     - Create a function `objective_rule(model)` to calculate the objective (expected return minus risk).
     - Use `pyo.Objective()` to set this function as the objective and specify maximization.
   - **Define Constraints:**
     - Create functions for each constraint (budget, scenario selection, probability).
     - Use `pyo.Constraint()` to add these constraints to the model.
   - **Solve the Model:**
     - Choose a solver (e.g., 'gurobi') using `pyo.SolverFactory()`.
     - Solve the model using `solver.solve(model)`.
   - **Print the Solution:**
     - Access and print the optimal values of variables and the objective function using `pyo.value()`.

Remember to install the necessary solver (e.g., Gurobi) and replace sample data with your actual data. 


## 4. Input problem data and try running the generated code

In [294]:
import pyomo.environ as pyo
import random

# Sample data for T and R_tj (You'll need to replace these with your actual data)
T = 5 
R = {(t,j): random.randint(50, 150) for t in range(1, T+1) for j in range(1, 11)}  
r = 0.05  # Sample risk aversion coefficient

# Model
model = pyo.ConcreteModel()

# Sets
model.J = pyo.RangeSet(1, 10)  # Strategies
model.I = pyo.RangeSet(1, 4)   # Budget scenarios
model.T = pyo.RangeSet(1, T)   # Time periods 

# Parameters
model.A = {j: random.randint(50, 150) for j in model.J}  # Investment cost for each strategy (sample data)
model.B = {1: 1000, 2: 1100, 3: 900, 4: 1200}  # Budget for each scenario
model.R = pyo.Param(model.T, model.J, initialize=R) # Returns
model.r = pyo.Param(initialize=r)

# Variables
model.x = pyo.Var(model.J, within=pyo.Binary)  # Investment decisions
model.y = pyo.Var(model.I, within=pyo.Binary)  # Budget scenario occurrence

# Objective function
def objective_rule(model):
    expected_return = sum(1/T * sum(model.R[t, j] * model.x[j] for j in model.J) for t in model.T)
    risk = model.r * sum(1/T * (sum(model.R[t, j] * model.x[j] for j in model.J) - expected_return)**2 for t in model.T)
    return expected_return - risk
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def budget_constraint(model, i):
    return sum(model.A[j] * model.x[j] for j in model.J) <= model.B[i] * model.y[i]
model.budget_constraint = pyo.Constraint(model.I, rule=budget_constraint)

def scenario_selection(model):
    return sum(model.y[i] for i in model.I) == 1
model.scenario_selection = pyo.Constraint(rule=scenario_selection)

def probability_constraint(model):
    return 0.55*model.y[1] + 0.4*model.y[2] + 0.04*model.y[3] + 0.01*model.y[4] <= 0.4
model.probability_constraint = pyo.Constraint(rule=probability_constraint)

# Solve
solver = pyo.SolverFactory('ipopt')  # Choose a solver you have installed
solver.solve(model)

# Output
print("Optimal Solution:")
for j in model.J:
    print(f"x[{j}] = {pyo.value(model.x[j])}")
for i in model.I:
    print(f"y[{i}] = {pyo.value(model.y[i])}")
print(f"Objective Value: {pyo.value(model.objective)}") 

Optimal Solution:
x[1] = 0.8674393130825288
x[2] = 0.0
x[3] = 0.0
x[4] = 0.0
x[5] = 0.0
x[6] = 0.8826378357043664
x[7] = 0.0
x[8] = 1.0
x[9] = 0.0
x[10] = 0.7079350806207348
y[1] = 0.2595019659236161
y[2] = 0.23591087811325903
y[3] = 0.28833551769183
y[4] = 0.21625163827129473
Objective Value: 374.76842070327336


## 5. Correct the code to verify model viability (optional)

In [295]:
# THE MODEL WAS RUN IN COLLAB WITH COUENNE AND THE RIGHT DATA TO VERIFY THE VIABILITY, BUT GAVE A WRONG OUTCOME.

#CODE OUTPUT:
# Saving investments_data.csv to investments_data (2).csv
# Optimal Solution:
# x[1] = 0.0
# x[2] = 0.0
# x[3] = 0.0
# x[4] = 0.0
# x[5] = 0.0
# x[6] = 0.0
# x[7] = 0.0
# x[8] = 0.0
# x[9] = 0.0
# x[10] = 0.0
# y[1] = 0.0
# y[2] = 0.0
# y[3] = 1.0
# y[4] = 0.0
# Objective Value: 0.0

## 6. Print the responses

In [300]:
print(response.text)

## Variables:

*  **x<sub>j</sub>:** Binary variable indicating whether to invest in strategy j (1) or not (0), where j = 1, 2, ..., 10.
* **y<sub>i</sub>:** Binary variable indicating whether budget scenario i occurs (1) or not (0), where i = 1, 2, 3, 4. 



In [301]:
print(response2.text)

$$\max  \sum_{t=1}^{T} \frac{1}{T} [\sum_{j=1}^{10}  (R_{tj} * x_j)] - r *  \sum_{t=1}^{T} \frac{1}{T}  [(\sum_{j=1}^{10}  (R_{tj} * x_j) - \sum_{t=1}^{T} \frac{1}{T} [\sum_{j=1}^{10}  (R_{tj} * x_j)])^2]  $$ 



In [302]:
print(response3.text)

## Constraints:

1. **Budget Constraint:**
   -  For each scenario i: 
      ∑<sub>j=1</sub><sup>10</sup> A<sub>j</sub> * x<sub>j</sub>  ≤  B<sub>i</sub> * y<sub>i</sub>  
      where B<sub>i</sub> represents the budget for scenario i (1000, 1100, 900, 1200).

2. **Scenario Selection:** Only one budget scenario can occur:
   - ∑<sub>i=1</sub><sup>4</sup> y<sub>i</sub> = 1

3. **Investment Uniqueness:** Invest in each strategy only once:
   - x<sub>j</sub> ∈ {0, 1} for j = 1, 2, ..., 10

4. **Scenario Probability:** Ensure the probability of exceeding the budget is within the tolerable limit:
   - 0.55y<sub>1</sub> + 0.4y<sub>2</sub> + 0.04y<sub>3</sub> + 0.01y<sub>4</sub>  ≤  0.4 

5. **Scenario Binary:**
   - y<sub>i</sub> ∈ {0, 1} for i = 1, 2, 3, 4 



In [303]:
print(response4.text)

```python
import pyomo.environ as pyo
import random

# Sample data for T and R_tj (You'll need to replace these with your actual data)
T = 5 
R = {(t,j): random.randint(50, 150) for t in range(1, T+1) for j in range(1, 11)}  
r = 0.05  # Sample risk aversion coefficient

# Model
model = pyo.ConcreteModel()

# Sets
model.J = pyo.RangeSet(1, 10)  # Strategies
model.I = pyo.RangeSet(1, 4)   # Budget scenarios
model.T = pyo.RangeSet(1, T)   # Time periods 

# Parameters
model.A = {j: random.randint(50, 150) for j in model.J}  # Investment cost for each strategy (sample data)
model.B = {1: 1000, 2: 1100, 3: 900, 4: 1200}  # Budget for each scenario
model.R = pyo.Param(model.T, model.J, initialize=R) # Returns
model.r = pyo.Param(initialize=r)

# Variables
model.x = pyo.Var(model.J, within=pyo.Binary)  # Investment decisions
model.y = pyo.Var(model.I, within=pyo.Binary)  # Budget scenario occurrence

# Objective function
def objective_rule(model):
    expected_return = sum(1/T * sum(model.R[